In [ ]:


# Hyperparameters
img_size = (48, 48)
batch_size = 64
num_classes = 7  # Number of emotion categories

# Split dataset into train and test
# Define the dataset directory and split ratios
data_dir = "/path/to/dataset"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
split_ratio = 0.8  # 80% train, 20% test

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Function to split data
def split_data(data_dir, train_dir, test_dir, split_ratio):
    for emotion in os.listdir(data_dir):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.isdir(emotion_path):
            continue

        # Create subdirectories in train and test folders
        os.makedirs(os.path.join(train_dir, emotion), exist_ok=True)
        os.makedirs(os.path.join(test_dir, emotion), exist_ok=True)

        # Get all files in the emotion folder
        files = os.listdir(emotion_path)
        train_files, test_files = train_test_split(files, test_size=1 - split_ratio, random_state=42)

        # Move files to train and test folders
        for file in train_files:
            shutil.copy2(os.path.join(emotion_path, file), os.path.join(train_dir, emotion, file))
        for file in test_files:
            shutil.copy2(os.path.join(emotion_path, file), os.path.join(test_dir, emotion, file))

# Split the dataset
split_data(data_dir, train_dir, test_dir, split_ratio)

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    color_mode="grayscale",
)

val_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    color_mode="grayscale",
)

# Build the Model
model = Sequential([
    Conv2D(64, (3, 3), activation="relu", input_shape=(img_size[0], img_size[1], 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Conv2D(256, (3, 3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Flatten(),

    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation="softmax"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-5)

# Training
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
)

# Save the model
model.save("emotion_classification_model.h5")

# Evaluate
results = model.evaluate(val_generator)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")
